In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [9]:
URLS = pd.read_csv('all_fight_decision_links.csv')

In [18]:
def grab_info_from_link(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    
    
    fighter_one = soup.find_all('a', href=lambda x: x and x.startswith('fighter'))[0].get_text(strip=True).replace(u'\xa0',' ')
    fighter_two = soup.find_all('a', href=lambda x: x and x.startswith('fighter'))[1].get_text(strip=True).replace(u'\xa0',' ')


    #Date stuff
    td_element = soup.find('td', class_='decision-top2')
    if td_element:
        # Extract text and split by line breaks
        text_lines = td_element.get_text(separator="\n").split("\n")
        # Assuming the date is always the line after the event title
        date_line = text_lines[4] if len(text_lines) > 1 else "Date not found"
        # Clean up any excess whitespace
        date = " ".join(date_line.split())
        # print(date)

    # Judge Stuff
    judges = soup.find_all('td', class_='judge')
    # Extract the names of the judges
    judge_names = [judge.get_text().replace(u'\xa0', u' ') for judge in judges]  # Replace non-breaking spaces with regular spaces
    judge_names = [judge_names[i].replace('\n', '') for i in range(len(judges))]
    judge_names


    #score stuff
    scores_dict = {}

    # Iterate over each judge's table
    judges_tables = soup.find_all('table', style="border-spacing: 1px; width: 100%")
    for judge_index, table in enumerate(judges_tables, start=1):
        # Get all the score rows for this judge
        score_rows = table.find_all('tr', class_='decision')
        for row in score_rows:
            round_number, score_fighter1, score_fighter2 = [td.get_text() for td in row.find_all('td', class_='list')]
            scores_dict[f"Judge {judge_index} Fighter 1 Round {round_number}"] = [score_fighter1]
            scores_dict[f"Judge {judge_index} Fighter 2 Round {round_number}"] = [score_fighter2]

    # Create a DataFrame
    scores_df = pd.DataFrame(scores_dict)
    scores_df['Date'] = date
    scores_df['Fighter 1'] = fighter_one
    scores_df['Fighter 2'] = fighter_two
    scores_df['Judge 1'] = judge_names[0]
    scores_df['Judge 2'] = judge_names[1]
    scores_df['Judge 3'] = judge_names[2]
    return scores_df

In [19]:
all_info = pd.DataFrame()
i=1


In [29]:
for url in URLS['0'][3150:]:
    all_info = pd.concat([all_info,grab_info_from_link(url)])
    i+=1
    if(i%10==0):
        print(i)

3160


IndexError: list index out of range

In [33]:
all_info = all_info.drop_duplicates()

In [41]:
all_info = all_info[all_info['Judge 1 Fighter 1 Round 1'].isin(['10','9','8','7'])]

In [44]:
all_info.to_csv('Fight_decision_data.csv')

In [45]:
all_info

,Judge 1 Fighter 1 Round 1,Judge 1 Fighter 2 Round 1,Judge 1 Fighter 1 Round 2,Judge 1 Fighter 2 Round 2,Judge 1 Fighter 1 Round 3,Judge 1 Fighter 2 Round 3,Judge 1 Fighter 1 Round 4,Judge 1 Fighter 2 Round 4,Judge 1 Fighter 1 Round 5,Judge 1 Fighter 2 Round 5,...,Judge 3 Fighter 1 Round 4,Judge 3 Fighter 2 Round 4,Judge 3 Fighter 1 Round 5,Judge 3 Fighter 2 Round 5,Date,Fighter 1,Fighter 2,Judge 1,Judge 2,Judge 3
0,9,10,9,10,10,9,10,9,10,9,...,10,9,10,9,"February 10, 2024",Jack Hermansson,Joe Pyfer,Michael Bell,Sal D'Amato,Chris Lee
0,10,9,10,9,10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"February 10, 2024",Ihor Potieria,Robert Bryczek,Adalaide Byrd,Chris Flores,Jacob Montalvo
0,10,9,10,9,10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"February 10, 2024",Michael Johnson,Darrius Flowers,Eric Colón,Junichiro Kamijo,Chris Lee
0,10,9,9,10,10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"February 10, 2024",Bolaji Oki,Timothy Cuamba,Michael Bell,Chris Leben,Chris Lee
0,10,9,9,10,10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"February 10, 2024",Loma Lookboonmee,Bruna Brasil,Eric Colón,Sal D'Amato,Junichiro Kamijo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,10,9,10,9,10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"January 19, 2008",Paul Kelly,Paul Taylor,Jeff Mullen,Chris Watts,Tony Weeks
0,9,10,10,8,10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"January 19, 2008",Sam Stout,Per Eklund,Douglas Crosby,Chris Watts,Tony Weeks
0,10,9,10,9,10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"August 25, 2007",Joe Stevenson,Kurt Pellegrino,Douglas Crosby,Gene LeBell,Cecil Peoples
0,10,9,10,9,10,9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"April 21, 2007",Andrei Arlovski,Fabricio Werdum,Douglas Crosby,Jeff Mullen,Cecil Peoples
